In [1]:
%matplotlib inline

import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Code hider, source: http://chris-said.io/2016/02/13/how-to-make-polished-jupyter-presentations-with-optional-code-visibility/
from IPython.display import HTML

HTML('''
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
''')

# Measuring Relationships and their Meaning from Phone Data

## Motivation

### Field domain
- social relationships and communication tie strength
- passive mobile device sensing
- ML for life-facet prediction (Zimmerman group)

## The gap we fill

- use of additional phone sensor data to contextualize communication
- user of AutoML to remove engineering bias in model choice/performance tuning
- finer grained social relationship role prediction

## Methods: raw features

### Communication logs


### Contact type

### Demographics

### Semantic location

## Methods: feature extraction

### Intensity and regularity

### Temporal tendency

### Channel selection

### Avoidance

## Methods: AutoML



## Results: 6-class relationship prediction



## Results: contact type analysis

## Results: 4-class relationship prediction

# Outline

- intro/motivation
- methods
    - raw data features
    - feature extraction
    - autoML
- results
    - "original" task: confusion matrix, baseline, etc
    - communication label analysis: why the confusions are made
    - "collapsed" task: confusion matrix, baseline, etc
    - analysis of EMC results
- discussion
    - what we've learned about social support
    - caveats to autoML:
        - no block feature selection
        - no variable importance implemented (yet)
    - size of data:
        - can run additional analyses with round 2 of LifeSense study